#1. Import Necessary Libraries



In [1]:
import gym
import numpy as np
import time
from IPython import display
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

#2. Support Function


In [2]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [3]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')
    return success/max_episodes 

In [4]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [5]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [6]:
runtime_data = []
winrate_data = []

#3. Value Iteration


In [7]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [8]:
runtimes = []
winrates = []

##3.1. FrozenLake-v1

In [9]:
env = gym.make('FrozenLake-v1')

start = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime: ", end-start)

optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes.append(end-start)
winrates.append(winrate)

Converged at 79-th iteration.
Runtime:  0.04733848571777344
Number of successes: 717/1000
Average number of steps: 37.07670850767085


##3.2. FrozenLake8x8-v1

In [10]:
env = gym.make('FrozenLake8x8-v1')

start = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime: ", end-start)

optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
winrate= play_multiple_times(env, optimal_policy, 1000)

runtimes.append(end-start)
winrates.append(winrate)

Converged at 117-th iteration.
Runtime:  0.21391034126281738
Number of successes: 714/1000
Average number of steps: 72.91736694677871


##3.3. Taxi-v3

In [11]:
env = gym.make('Taxi-v3')

start = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime: ", end-start)

optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes.append(end-start)
winrates.append(winrate)

Converged at 116-th iteration.
Runtime:  3.180743932723999
Number of successes: 1000/1000
Average number of steps: 13.096


In [12]:
runtime_data.append(runtimes)
winrate_data.append(winrates)

#4. Policy Iteration

In [13]:
def policy_iteration(env, max_iters, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    for i in range(max_iters):
        prev_policy = np.copy(policy)

        # policy evaluation
        v_values = policy_evaluation(env, policy, max_iters=max_iters, gamma=gamma)
        
        # policy improvement
        policy = policy_extraction(env, v_values, gamma=gamma)

        # check policy convergence
        flag = True
        for state in range(env.observation_space.n):
            if prev_policy[state] != policy[state]:
                flag = False
                break
        
        if flag == True:
            print(f'Converged at {i}-th iteration.')
            break

    return policy

In [14]:
runtimes = []
winrates = []

##4.1. FrozenLake-v1

In [15]:
env = gym.make('FrozenLake-v1')

start = time.time()
optimal_policy = policy_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime: ", end-start)

winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes.append(end-start)
winrates.append(winrate)

Converged at 5-th iteration.
Runtime:  0.04864358901977539
Number of successes: 716/1000
Average number of steps: 37.54608938547486


##4.2. FrozenLake8x8-v1

In [16]:
env = gym.make('FrozenLake8x8-v1')

start = time.time()
optimal_policy = policy_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime: ", end-start)

winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes.append(end-start)
winrates.append(winrate)

Converged at 9-th iteration.
Runtime:  0.5882265567779541
Number of successes: 733/1000
Average number of steps: 70.02864938608458


##4.3. Taxi-v3

In [17]:
env = gym.make('Taxi-v3')

start = time.time()
optimal_policy = policy_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime: ", end-start)

winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes.append(end-start)
winrates.append(winrate)

Converged at 16-th iteration.
Runtime:  11.351135015487671
Number of successes: 1000/1000
Average number of steps: 13.09


In [18]:
runtime_data.append(runtimes)
winrate_data.append(winrates)

#5. Conclusion

In [19]:
runtime_df = pd.DataFrame(runtime_data, columns=['FrozenLake-v1', 'FrozenLake8x8-v1', 'taxi-v3'], 
                          index=['Value Iteration', 'Policy Iteration'])
winrate_df = pd.DataFrame(winrate_data, columns=['FrozenLake-v1', 'FrozenLake8x8-v1', 'taxi-v3'], 
                          index=['Value Iteration', 'Policy Iteration'])
runtime_df.columns = pd.MultiIndex.from_product([['RUNTIME'], runtime_df.columns.tolist()])
winrate_df.columns = pd.MultiIndex.from_product([['WINRATE'], winrate_df.columns.tolist()])
display.display(runtime_df)
display.display(winrate_df)

RUNTIME                            
                 FrozenLake-v1 FrozenLake8x8-v1    taxi-v3
Value Iteration       0.047338         0.213910   3.180744
Policy Iteration      0.048644         0.588227  11.351135

WINRATE                         
                 FrozenLake-v1 FrozenLake8x8-v1 taxi-v3
Value Iteration          0.717            0.714     1.0
Policy Iteration         0.716            0.733     1.0

In [20]:
#   Nhìn chung cả hai thuật toán đều đảm bảo hội tụ về một phương án tối ưu. 
#   Trong cả 3 games, Policy Iteration đều hội tụ về phương án tối ưu trong rất ít vòng lặp so với Value Iteration.
#   Tuy nhiên thời gian tìm lời giải của Policy Iteration lâu hơn so với Value Iteration. Lý do 
#   là vì Policy Iteration tính quá nhiều lần v_value trong mỗi vòng lặp